In [1]:
!pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tomli-2.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-macosx_10_12_x86_64.whl.metadata (4.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 45.5 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 60.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.2 MB/s  0:00:00
Using cached hf_xet-1.2.0-cp37-abi3-macosx_10_12_x86_64.whl (2.9 MB)
Using cached tomli-2.3.0-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.8 MB/s  0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=e8d076806c1b1e6b47709b040b5c99ddfbb25aa2

In [2]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

/Users/sulbhajain/opt/anaconda3/envs/azure_agent_py310/lib/python3.10/site-packages/autogen_ext/models/azure/_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


### Vector-store initialization

In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

/Users/sulbhajain/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 29.4MiB/s]


In [ ]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2,
            
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

In [9]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

In [10]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'type': 'explanation', 'source': 'training'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'type': 'explanation', 'source': 'training'}


Weather Information for london:
Weather for London:
T